In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model = KeyedVectors.load_word2vec_format('50thousand.txt')

In [ ]:
# 定义需要计算相似性的词汇
electrodes = ['carbon','graphitic','CNTs','RuO2','TiO2','PANI','polypyrrole']
electrolytes = ['H2SO4','KCl','EMIMBF4']
separator = ['cellulose','polypropylene']
enclosure = ['Al','steel']
performance = ['capacitance','resistance']
structure = ['nanoporous', 'microporous','nanostructure','symmetric','asymmetric']
characterization = ['CV','EIS','GCD','BET','SEM','TEM','XPS']
target_words = ['electrodes','electrolytes','separator','enclosure','performance','structure','characterization']

# 计算余弦相似性
similarities = {}

# 计算各类别词汇与目标词汇的相似性
categories = {'electrodes':electrodes,'electrolytes':electrolytes,'separator':separator,'enclosure':enclosure,'performance':performance,'structure':structure,'characterization':characterization}

for target_word, category in categories.items():
    if target_word in target_words:
        for word in category:
            if word in model and target_word in model:
                similarities[(word, target_word)] = model.similarity(word, target_word)

# # 计算target_words之间的相似性
# for i in range(len(target_words)):
#     for j in range(i + 1, len(target_words)):
#         word1 = target_words[i]
#         word2 = target_words[j]
#         if word1 in model and word2 in model:
#             similarities[(word1, word2)] = model.similarity(word1, word2)

# 计算Li-ion与target_words之间的相似性
li_ion_similarities = {}
if 'supercapacitors' in model:
    for word in target_words:
        if word in model:
            li_ion_similarities[(word, 'supercapacitors')] = model.similarity(word, 'supercapacitors')

# 创建图
G = nx.Graph()

# 添加节点，包括中心节点Li-ion
all_words = electrodes + electrolytes + separator + enclosure + performance + structure + characterization + target_words + ['supercapacitors']
for word in all_words:
    G.add_node(word)

# 添加边并设置宽度
edges = []
for (word1, word2), sim in similarities.items():
    G.add_edge(word1, word2, weight=sim * 10)  # 乘以10使边的宽度更明显
    edges.append((word1, word2))

# 将Li-ion节点与target_words连接
for (word, li_ion), sim in li_ion_similarities.items():
    G.add_edge(word, li_ion, weight=sim * 10)  # 设置权重

# 绘制图
plt.figure(figsize=(18, 18), dpi=1000)
pos = nx.spring_layout(G, seed=42, k=0.5)  # 增加k值以增大节点间距

# 根据权重绘制边
weights = [G[u][v]['weight'] for u, v in edges]
li_ion_weights = [G[u][v]['weight'] for u, v in G.edges('supercapacitors')]

# 绘制边，设置边颜色和透明度
nx.draw_networkx_edges(G, pos, edgelist=edges, width=[weight * 2 for weight in weights], edge_color='plum', alpha=0.3)  # 乘以2增加宽度
nx.draw_networkx_edges(G, pos, edgelist=[(word, 'supercapacitors') for word in target_words], width=[weight * 2 for weight in li_ion_weights], edge_color='plum', alpha=0.3)

# 为Li-ion设置浅绿色，为target_words设置灰色，其他节点颜色不变
node_color_map = []
for node in G:
    if node == 'supercapacitors':
        node_color_map.append('lightgreen')
    elif node in target_words:
        node_color_map.append('powderblue')
    else:
        node_color_map.append('mistyrose')  # 其他节点颜色

# 绘制节点和标签
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color=node_color_map)  
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')

plt.title('Knowledge Graph Based on Cosine Similarity', fontsize=30)
plt.axis('off')  # 关闭坐标轴

plt.savefig('supercapacitors-knowledge-graph-.png')
plt.show()